In [ ]:
from typing import Dict, TypedDict,Annotated
from pydantic import Field, BaseModel, ValidationError
from dotenv import load_dotenv
from langchain_teddynote import logging
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START,END
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
logging.langsmith("CH17-LangGraph-Modules")

In [ ]:
memory = MemorySaver()

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
tool = TavilySearch(max_results=3)
tools = [tool]
llm = ChatOpenAI(model="gpt-4o-mini")
# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)

def chatbot(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder = StateGraph(State)
graph_builder.add_node('chatbot',chatbot)
tool_node = ToolNode(tools)
graph_builder.add_node('tools',tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)


In [ ]:
# 그래프 빌더 컴파일
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from langchain_teddynote.graphs import visualize_graph

# 그래프 시각화
visualize_graph(graph)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [ ]:
# 첫 질문
question = (
    "2025년 9월 18일 기준 삼성전자 주식 종가"
)
graph.invoke({"messages": [("user", question)]}, config=config)
# for event in graph.stream({"messages": [("user", question)]}, config=config):
#     for value in event.values():
#         value["messages"][-1].pretty_print()

## 두개이상 ToolNode 활용방법

In [ ]:
from langchain_core.tools import tool
from langchain_teddynote.tools import GoogleNews
from typing import List, Dict

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import AIMessage



In [ ]:

class local_search:
    """
    RAG 기반 정보를 검색 결과를 반환 하는 클래스 입니다.
    """
    def __init__(self):
        """
        GoogleNews 클래스를 초기화합니다.
        base_url 속성을 설정합니다.
        """
        self.loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
        self.text_spitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        self.split_docs = self.loader.load_and_split(self.text_spitter)

    def _get_embeddings(self):

        model_name = "intfloat/multilingual-e5-large-instruct"
        # model_name = "intfloat/multilingual-e5-large"
        hf_embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cuda"},  # cuda, cpu
            encode_kwargs={"normalize_embeddings": True},
        )
        return hf_embeddings

    def _query(self,keyword,vector):
        self.retriever = vector.as_retriever()
        return self.retriever.invoke(keyword)

    def search_by_keyword(self, keyword):
        """
        최신 뉴스를 검색합니다.

        Args:
            k (int): 검색할 뉴스의 최대 개수 (기본값: 3)

        Returns:
            List[Dict[str, str]]: URL과 내용을 포함한 딕셔너리 리스트
        """
        vector = FAISS.from_documents(documents=self.split_docs, embedding=self._get_embeddings())
        return self._query(keyword,vector)

 





In [ ]:

# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

@tool
def search_local_data(query: str) -> List[Dict[str, str]]:
    """Search pdf file infomation by input keyword"""
    news_tool = local_search()
    return news_tool.search_by_keyword(query)

In [ ]:

# 도구 리스트 생성
tools = [search_news, search_local_data]

# ToolNode 초기화
tool_node = ToolNode(tools)

In [ ]:
# 단일 도구 호출을 포함하는 AI 메시지 객체 생성
# AIMessage 객체이어야 함
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "search_local_data",  # 도구 이름
            "args": {"query": "삼성전자"},  # 도구 인자
            "id": "tool_call_id",  # 도구 호출 ID
            "type": "tool_call",  # 도구 호출 유형
        }
    ],
)

# 도구 노드를 통한 메시지 처리 및 날씨 정보 요청 실행
tool_node.invoke({"messages": [message_with_single_tool_call]})

In [ ]:
# 단일 도구 호출을 포함하는 AI 메시지 객체 생성
# AIMessage 객체이어야 함
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "search_news",  # 도구 이름
            "args": {"query": "삼성전자"},  # 도구 인자
            "id": "tool_call_id",  # 도구 호출 ID
            "type": "tool_call",  # 도구 호출 유형
        }
    ],
)

# 도구 노드를 통한 메시지 처리 및 날씨 정보 요청 실행
tool_node.invoke({"messages": [message_with_single_tool_call]})

## 두개 agent 중 알아서 사용

In [ ]:
# 다중 도구 호출을 포함하는 AI 메시지 객체 생성 및 초기화
# message_with_multiple_tool_calls = AIMessage(
#     content="",
#     tool_calls=[
#         {
#             "name": "search_news",
#             "args": {"query": "삼성전자 주가"},
#             "id": "tool_call_id",
#             "type": "tool_call",
#         },
# {
#             "name": "search_local_data",  # 도구 이름
#             "args": {"query": "삼성전자 생성형 AI"},  # 도구 인자
#             "id": "tool_call_id",  # 도구 호출 ID
#             "type": "tool_call",  # 도구 호출 유형
#         }
#     ],
# )

# 생성된 메시지를 도구 노드에 전달하여 다중 도구 호출 실행
# tool_node.invoke({"messages": [message_with_multiple_tool_calls]})

In [ ]:
model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [ ]:
model_with_tools

In [ ]:
model_with_tools.invoke("처음 5개의 소수를 출력하는 python code 를 작성해줘").tool_calls

In [ ]:
# 도구 노드를 통한 메시지 처리 및 LLM 모델의 도구 기반 응답 생성
tool_node.invoke(
    {
        "messages": [
            model_with_tools.invoke(
                "처음 5개의 소수를 출력하는 python code 를 작성해줘"
            )
        ]
    }
)

In [ ]:
tool_node

## Agent 와 함께 사용하기 
langGraph 내에서 Toolnode 사용

In [ ]:
# LangGraph 워크플로우 상태 및 메시지 처리를 위한 타입 임포트
from langgraph.graph import StateGraph, MessagesState, START, END


# LLM 모델을 사용하여 메시지 처리 및 응답 생성, 도구 호출이 포함된 응답 반환
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


# 메시지 상태 기반 워크플로우 그래프 초기화
workflow = StateGraph(MessagesState)

# 에이전트와 도구 노드 정의 및 워크플로우 그래프에 추가
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 워크플로우 시작점에서 에이전트 노드로 연결
workflow.add_edge(START, "agent")

# 에이전트 노드에서 조건부 분기 설정, 도구 노드 또는 종료 지점으로 연결
workflow.add_conditional_edges("agent", tools_condition)

# 도구 노드에서 에이전트 노드로 순환 연결
workflow.add_edge("tools", "agent")

# 에이전트 노드에서 종료 지점으로 연결
workflow.add_edge("agent", END)


# 정의된 워크플로우 그래프 컴파일 및 실행 가능한 애플리케이션 생성
app = workflow.compile()

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(app)

In [ ]:
# 검색 질문 수행
for chunk in app.stream(
    {"messages": [("human", "대한민국 수도는?")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 도구 호출이 필요 없는 질문 수행
for chunk in app.stream(
    {"messages": [("human", "안녕? 반가워")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()